# Tutorial 1. Gradient descent optimization

In this tutorial, we will learn:

* how to optimize a function using the gradient descent method

In [1]:
import sys
import cmath
import math
import os

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
import util.libutil as comn
from libra_py import data_outs

/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<float, std::all

## 1. Theory

The approach is pretty straightforward - to reach the minimum of the function, we just need to follow oppositely to its gradient, so the update steps are:

$$
q_\alpha (n+1) = q_\alpha (n) - \frac{\partial f}{\partial q_\alpha} (q_n) * \Delta
$$

Here, $\Delta$ is the step size, $\alpha$ is index of the degree of freedom (DOF), and $q$ refers to a multidimensional coordinates on which the function depends.

Obviously, the approach requires a choice of some starting point, $q(0)$, which may be non-trivial, especially for fast varying functions (with multiple minima) and in high dimensions.

The choice of the stepping parameter $\Delta$ is another question - it shall be neither too large (so the procedure does converge), nor too small (so it converges efficiently)


The algorithm is available in Libra via the function 

`grad_descent(grad_function, dof, funct_params, grad_tol, step_size, max_steps)` 

defined in the **opt** library. The function takes the following parameters:


  * `grad_function` - shoud be Python function (representing $f$ in the above theory) of the signature
    `py_funct(MATRIX& dof, bp::object params)` 
    This function computes the gradient of thefunction which we minimize and should return it as a *MATRIX* type
      
    Here:
      - `dof` is of type *MATRIX* and represents the arguments of the function ($q$ in the above theory)
      - `params` is of type Python object (a function, a dictionary, a list, a class, etc), which stores
          the parameters of the function (so it is advisable to use a Python dictionary for it)
          
  * `dof` - represents the arguments of the function and should be of type *MATRIX*. Note that this variable
    would contain the final results of the calculations, so the end and starting values may be different.
    
  * `funct_params` - should be of the *Python object* type and contains the parameters passed to the `grad_function`

  * `grad_tol` - the tolerance of the gradient value. It is one of the stopping critetion: the iterations stop as soon ass the max value of a projection of the gradient $ \frac{\partial f}{\partial q_\alpha} (q_n) $ vector drops below this level
  
  * `step_size` - is the algorithm parameter, which corresponds to $\Delta$ in the equation above
  
  * `max_steps` - is yet another stopping criterion. The iterations stop after that many steps, no matter whether the gradient criterion is met or not.
  
  
## 2. Examples

In all cases below, we shall define our function to optimize. We do this in accordiance with the expectation from the above function signatures. Then we essentially can the `grad_descent` function with the suitable parameters and get the results.

### 2.1. Example 1.

Here is our model function of 3 variables (3 DOFs):

$$ E(x,y,z) = \frac{1}{2} (x-1)^2 + (y+2)^2 + z^2 $$

To match the format, we represent the variables $x, y, z$ as a 3D vector:  


$$
q = \begin{pmatrix}
x \\
y \\
z \\
\end{pmatrix}
$$

It is clear that the minimum of this function will be at $q = (1, -2, 0)^T$

The gradients of the function are given by:

$$
\nabla E = \begin{pmatrix}
\frac{\partial E}{\partial x} \\
\frac{\partial E}{\partial y} \\
\frac{\partial E}{\partial z} \\
\end{pmatrix} = 
\begin{pmatrix}
x-1 \\
2(y+2) \\
2z \\
\end{pmatrix}
$$

This encoded in the following function:

In [2]:
def model1(q, params):
    """
    q = (x,y,z)
    E = 1/2* ((x-1)^2 + (y+2)^2 + z^2)

    """

    grd = MATRIX(3,1) 
    x,y,z = q.get(0), q.get(1), q.get(2)

    grd.set(0, x-1.0)
    grd.set(1, y+2.0)
    grd.set(2, z)

    return grd

Now, we can define a test function that would initialize the guessed minimum, at $q = (1, 1, 1)^T$ and would make 10 steps of relative size 1.0 and print out the resulting values of the guess minima.

Note that in this example, the function doesn't take any parameters, but we still need to satisfy the functions signatures, so we pass an empty dictionary of parameters.

This function prints out the resulting matrix to the Jupyter output via the use of `data_outs.print_matrix` functions. The matrices can also be printed out to the terminal (see behind the Jupyter, perhaps) with the member function of the `MATRIX` class, the `show_matrix` function.

In [3]:
def test():
    q = MATRIX(3,1)
    q.set(0, 1)
    q.set(1, 1)
    q.set(2, 1)

    params = {}    
           
    qopt = grad_descent(model1, q, params, 1e-6, 1.0, 10)

    print("The optimized positions are:"); data_outs.print_matrix(qopt)
    qopt.show_matrix()

Note how the above function didn't produce anything yet - since it is just a definition.

And here we call the computations:

In [4]:
test()

The optimized positions are:
1.0  
-2.0  
0.0  


This is exactly what we expected to see

### 2.2. Example 2

Now, lets modify our function above to do just 1 step per function call, print out the information more frequently so that we could see how the optimization takes place.

In [11]:
def test2():
    q = MATRIX(3,1)
    q.set(0, 1)
    q.set(1, 1)
    q.set(2, 1)

    params = {}    
        
    tol = 1e-5
    err = 2*tol
    i = 0
    
    while err > tol and i<50:
            
        q = grad_descent(model1, q, params, 1e-6, 0.5, 1)
        
        grd = model1(q, params)        
        err = max( abs(grd.get(0)), abs(grd.get(1)), abs(grd.get(2)))
        print(F"step {i} gradient = {grd.get(0), grd.get(1), grd.get(2) } \
        The optimized positions are: {q.get(0), q.get(1), q.get(2)}")
        
        i += 1 

In [12]:
test2()

step 0 gradient = (0.0, 1.5, 0.5)         The optimized positions are: (1.0, -0.5, 0.5)
step 1 gradient = (0.0, 0.75, 0.25)         The optimized positions are: (1.0, -1.25, 0.25)
step 2 gradient = (0.0, 0.375, 0.125)         The optimized positions are: (1.0, -1.625, 0.125)
step 3 gradient = (0.0, 0.1875, 0.0625)         The optimized positions are: (1.0, -1.8125, 0.0625)
step 4 gradient = (0.0, 0.09375, 0.03125)         The optimized positions are: (1.0, -1.90625, 0.03125)
step 5 gradient = (0.0, 0.046875, 0.015625)         The optimized positions are: (1.0, -1.953125, 0.015625)
step 6 gradient = (0.0, 0.0234375, 0.0078125)         The optimized positions are: (1.0, -1.9765625, 0.0078125)
step 7 gradient = (0.0, 0.01171875, 0.00390625)         The optimized positions are: (1.0, -1.98828125, 0.00390625)
step 8 gradient = (0.0, 0.005859375, 0.001953125)         The optimized positions are: (1.0, -1.994140625, 0.001953125)
step 9 gradient = (0.0, 0.0029296875, 0.0009765625)         The 

Also, this example allows you to introduce more control of the stopping of the optimization process. Note that the tolerance defined by the `tol` variable matters only in our Python loop, but the tolerance passed to the function `grad_descent` doesn't matter for 1 step.

See how even though we considered doing up to 50 steps, we have met the convergence criterion a bit earlier and exited our loop. 